In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path

In [2]:
def files2df_full(path):
    all_files = glob.glob(path)
    listu = []
    for filename in all_files:
        try:
            df = pd.read_csv(filename, index_col=None, header=0, sep='\t')
            splitted_name = os.path.basename(filename).split('_')
            name = splitted_name[2]
            df['assay'] = name
            listu.append(df)
        except:
            print(filename)
    frame = pd.concat(listu, axis=0, ignore_index=True)
    return frame

In [4]:
evaluated_models_df = files2df_full('evaluated_models/*.tsv')
evaluated_models_df.sampling = evaluated_models_df.sampling.fillna('original')
evaluated_models_df['sampling'] = evaluated_models_df['sampling'].replace(['naive'], 'original')

In [5]:
algorithm = []
for row in evaluated_models_df['model']:
    if row in ['ada', 'gbm', 'LogitBoost', 'xgbDART', 'xgbTree']:
        algorithm.append('boosting')
    elif row in ['AdaBag', 'treebag']:
        algorithm.append('bagging')
    elif row in ['knn', 'kknn']:
        algorithm.append('kNN')
    elif row in ['lda']:
        algorithm.append('LDA')
    elif row in ['bayes']:
        algorithm.append('NB')
    elif row in ['ranger', 'Rborist', 'rf']:
        algorithm.append('RF')
    elif row in ['C5.0']:
        algorithm.append('C5.0')
    elif row in ['regLogistic']:
        algorithm.append('LR')
    elif row in ['svmLinearWeights']:
        algorithm.append('SVM')
    elif row in ['nnet', 'pcaNNet']:
        algorithm.append('ANN')
    else:
        algorithm.append('missing')
        
evaluated_models_df['algorithm'] = algorithm
evaluated_models_df.to_csv('all_models_evaluated.tsv', sep='\t', index=False)

In [6]:
# Selecting the models that have the lowest FPR at 90% of recall.
selected = evaluated_models_df[evaluated_models_df.groupby(['assay'])['FPR_09'].rank('dense', ascending=True) == 1]